In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
test = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
test.to_dataframe()

,num_rows
0,3233


In [3]:
q = """
SELECT (area_water_meters/(area_land_meters + area_water_meters)) * 100 AS water_area_percentage
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
"""
query_job = bq.query(q)
result = query_job.to_dataframe()

In [4]:
#q1
result['water_area_percentage'].iloc[0]

3.314489915768649

In [5]:
q2 = """
SELECT 
    s.state, COUNT(*) as county_count
FROM 
    `bigquery-public-data.geo_us_boundaries.counties` AS c
JOIN 
    `bigquery-public-data.geo_us_boundaries.states` AS s
ON 
    c.state_fips_code = s.state_fips_code
GROUP BY 
    s.state
ORDER BY 
    county_count DESC
LIMIT 5
"""
# Run the query
query_job = bq.query(q2)
result = query_job.to_dataframe()

In [6]:
#q2
dict(zip(result.state, result.county_count))

{'TX': 254, 'GA': 159, 'VA': 133, 'KY': 120, 'MO': 115}

In [7]:
iowaCost = 6.25
job_config = bigquery.QueryJobConfig(use_query_cache=False)

query_job_q1 = bq.query(q, job_config=job_config)
query_job_q2 = bq.query(q2, job_config=job_config)

cost_q1 = (query_job_q1.total_bytes_billed / (1024**4)) * iowaCost
cost_q2 = (query_job_q2.total_bytes_billed / (1024**4)) * iowaCost

cost_dict = {'q1': cost_q1, 'q2': cost_q2}

In [8]:
#q3
cost_dict

{'q1': 5.9604644775390625e-05, 'q2': 0.00011920928955078125}

In [9]:
from google.cloud import bigquery
from google.cloud.exceptions import Conflict

bq = bigquery.Client()
dataset_id = "cs544-s24-412418.p8"
dataset = bigquery.Dataset(dataset_id)

try:
    dataset = bq.create_dataset(dataset, exists_ok=True)
    print("Dataset {} created.".format(dataset_id))
except Conflict:
    print("Dataset already exists.")

table_id = "cs544-s24-412418.p8.hdma"
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET)
uri = "gs://raffanti/hdma-wi-2021-split.parquet"

load_job = bq.load_table_from_uri(
uri,
table_id,
job_config=job_config
)

load_job.result()
print("Table loaded successfully.")

Dataset cs544-s24-412418.p8 created.
Table loaded successfully.


In [10]:
#q4
datasets = [ds.dataset_id for ds in bq.list_datasets()]
datasets

['p8']

In [11]:
q5 = bq.query(
"""
SELECT
  c.county_name,
  COUNT(*) as loan_count
FROM
  `cs544-s24-412418.p8.hdma` h
JOIN
  `bigquery-public-data.geo_us_boundaries.counties` c
ON
  h.county_code = c.geo_id
GROUP BY
  c.county_name
ORDER BY
  loan_count DESC
LIMIT 10;
""")
q5ans = q5.to_dataframe()

In [12]:
#q5
dict(zip(q5ans.county_name, q5ans.loan_count))

{'Milwaukee': 232850,
 'Dane': 192785,
 'Waukesha': 170795,
 'Brown': 78075,
 'Racine': 65035,
 'Outagamie': 57615,
 'Kenosha': 53720,
 'Washington': 53630,
 'Rock': 49170,
 'Winnebago': 46550}

In [13]:
url = "https://docs.google.com/spreadsheets/d/1FfalqAWdzz01D1zIvBxsDWLW05-lvANWjjAj2vI4A04/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("cs544-s24-412418.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [14]:
q6 = bq.query(
"""
SELECT COUNT(*) as num_applications
FROM `cs544-s24-412418.p8.applications`
WHERE CAST(income AS STRING) = '100000'
""")

In [15]:
#q6
q6.to_dataframe()['num_applications'][0]

13

In [16]:
q7 = bq.query(
"""
WITH spatial_join AS (
  SELECT
    h.*,
    c.county_name
  FROM
    `cs544-s24-412418.p8.applications` h
  JOIN
    `bigquery-public-data.geo_us_boundaries.counties` c
  ON
    ST_WITHIN(ST_GEOGPOINT(h.longitude, h.latitude), c.county_geom)
  WHERE
    c.state_fips_code = '55'
)

SELECT
  county_name,
  COUNT(*) AS num_applications
FROM
  spatial_join
GROUP BY
  county_name;

""")
q7ans = q7.to_dataframe()

In [17]:
#q7
dict(zip(q7ans.county_name, q7ans.num_applications))

{'Fond du Lac': 1,
 'Dane': 17,
 'Oneida': 3,
 'Green Lake': 1,
 'Door': 7,
 'Walworth': 5,
 'Columbia': 1,
 'Jefferson': 2,
 'Brown': 2,
 'Bayfield': 1,
 'Outagamie': 1,
 'Sauk': 1,
 'Juneau': 1,
 'Douglas': 1,
 'Marinette': 3,
 'Florence': 1,
 'Milwaukee': 4,
 'Barron': 1,
 'Monroe': 2,
 'Kewaunee': 1,
 'Sheboygan': 1,
 'Winnebago': 1}

In [18]:
evaluate_model_query = """
SELECT
    mean_absolute_error
FROM
    ML.EVALUATE(MODEL `cs544-s24-412418.p8.machine`, (
        SELECT 
            loan_amount,
            income,
            loan_term
        FROM 
            `cs544-s24-412418.p8.hdma`
        WHERE 
            dataset = 'test'
    ))
"""
query_job = bq.query(evaluate_model_query)
result = query_job.to_dataframe()

In [19]:
#q8
result['mean_absolute_error'][0]

77312.06632498761

In [20]:
get_weight_query = """
SELECT
    processed_input,
    weight
FROM
    ML.WEIGHTS(MODEL `cs544-s24-412418.p8.machine`)
WHERE
    processed_input = 'loan_term'
"""
query_job = bq.query(get_weight_query)
result = query_job.to_dataframe()

In [21]:
#q9
result['weight'].iloc[0] 

380.24489177351376

In [22]:
%load_ext google.cloud.bigquery

In [23]:
%%bigquery predictQuery
SELECT *
FROM
  ML.PREDICT(MODEL `cs544-s24-412418.p8.machine`,
    (
    SELECT
      loan_amount, income, 360 AS loan_term
    FROM
      `cs544-s24-412418.p8.applications` 
    ))

Query is running:   0%|          |

Downloading:   0%|          |

In [24]:
#q10
q10 = (predictQuery['loan_amount'] > predictQuery['predicted_loan_amount']).sum()
q10 / len(predictQuery)

0.5955056179775281